# <font color = 'black'> Análise exploratória de dados de assiduidade em consultas na cidade de Vitória - ES. </font> 


#   <font color = 'blue'> Grupo 1: </font> 
* Aline Carvalho
* Camila Bianca
* Felipe Cunha
* Ivan Felipe 
* Mateus Brito
* Vinícius Peixoto

# <font color = 'blue'> Dicionário de Dados: </font> 

## Este conjunto de dados contém informações sobre as consultas médicas marcadas em clínicas no Brasil e se os pacientes compareceram ou não às consultas.

### O conjunto de dados inclui as seguintes colunas:

* PatientId: identificador único do paciente
* AppointmentID: identificador único da consulta
* Gender: gênero do paciente (Feminino/Masculino)
* ScheduledDay: data em que a consulta foi marcada
* AppointmentDay: data da consulta
* Age: idade do paciente
* Neighbourhood: nome do bairro onde a consulta ocorrerá
* Scholarship: indica se o paciente faz parte do programa Bolsa Família (0 = não, 1 = sim)
* Hipertension: indica se o paciente tem hipertensão (0 = não, 1 = sim)
* Diabetes: indica se o paciente tem diabetes (0 = não, 1 = sim)
* Alcoholism: indica se o paciente tem problemas com álcool (0 = não, 1 = sim)
* Handcap: indica se o paciente é deficiente (0 = não, 1 = sim)
* SMS_received: indica se o paciente recebeu uma mensagem de lembrete da consulta (0 = não, 1 = sim)
* No-show: indica se o paciente compareceu à consulta (No = sim, Yes = não)

##  <font color = 'blue'>Perguntas a serem respondidas:  </font>
### Ação combativa
- Qual o impacto do recebimento do SMS em cada grupo?
  - Geral 
  - Problemas de saúde
  - Idade
  - PCD

### Ação paliativa
- Qual a faixa etária mais assídua e a mais ausente? 
- Qual(is) o(s) bairro(s) de pessoas menos assíduas? 
- Quais os problemas de saúde das pessoas menos assíduas? 
- intervalo de tempo entre o dia da marcação e o dia da consulta em si.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm

tqdm.pandas(desc='progress')
warnings.filterwarnings("ignore")


In [ ]:
# fonte: https://www.kaggle.com/datasets/sarvaninandipati/brazils-medical-appointment-show-or-no-show
df = pd.read_excel('appointment_prediction_data.xlsx')

# fonte: Vitória (Censo e UDH) - Atlas do Desenvolvimento Humano no Brasil - Ipea (https://dados.gov.br/dados/conjuntos-dados/atlasbrasil)
aux = pd.read_csv('RM_63200_Vitória_Base_UDH_2000_2010.csv')

In [ ]:
def uppercase(x):
    return str(x).upper()

# usado somente nome dos bairros e a renda per capta deles
aux['NOME_UDH'] = aux['NOME_UDH'].progress_apply(lambda x: uppercase(x))
dados_renda_per_capta = aux[['NOME_UDH', 'RDPC']]
dados_renda_per_capta = dados_renda_per_capta.groupby('NOME_UDH').mean().reset_index()

# fonte: Plano Municipal de Saúde 2022-2025 Vitória - 2022 (https://m.vitoria.es.gov.br/download.php?tipo=1&id=3521)
regioes_saude = {
    'CONTINENTAL': ['AEROPORTO','ANTÔNIO HONÓRIO','BOA VISTA','JABOUR','JARDIM CAMBURI',
                    'JARDIM DA PENHA','MARIA ORTIZ','MATA DA PRAIA','MORADA DE CAMBURI',
                    'PONTAL DE CAMBURI','REPÚBLICA','SEGURANÇA DO LAR','SOLON BORGES',
                    'GOIABEIRAS', 'PARQUE INDUSTRIAL'],
    'CENTRO': ['CENTRO','DO CABRAL','DO MOSCOSO','DO QUADRO','FONTE GRANDE',
                     'ILHA DO PRÍNCIPE','PARQUE MOSCOSO','PIEDADE','SANTA CLARA',
                     'SANTA TEREZA','VILA RUBIM'],
    'SANTO ANTÔNIO': ['ARIOVALDO FAVALESSA','BELA VISTA','CARATOÍRA','ESTRELINHA',
                      'GRANDE VITÓRIA','INHANGUETÁ','MÁRIO CYPRESTE','SANTO ANTÔNIO',
                      'UNIVERSITÁRIO'],
    'MARUÍPE': ['ANDORINHAS','BONFIM','CONSOLAÇÃO','DA PENHA','DE LOURDES','GURIGICA',
                'HORTO','ITARARÉ','JOANA D\'ARC','MARUÍPE','SANTA CECÍLIA','SANTA MARTHA',
                'SANTOS DUMONT','SÃO BENEDITO','SÃO CRISTÓVÃO','TABUAZEIRO'],
    'SÃO PEDRO': ['COMDUSA','CONQUISTA','ILHA DAS CAIEIRAS','NOVA PALESTINA','REDENÇÃO',
                  'RESISTÊNCIA','SANTO ANDRÉ','SANTOS REIS','SÃO JOSÉ','SÃO PEDRO'],
    'FORTE SÃO JOÃO': ['BARRO VERMELHO','BENTO FERREIRA','CRUZAMENTO','ENSEADA DO SUÁ',
                       'FORTE SÃO JOÃO','FRADINHOS','MONTE BELO','ILHA DE SANTA MARIA',
                       'ILHA DO BOI','ILHA DO FRADE','JESUS DE NAZARETH','JUCUTUQUARA',
                       'NAZARETH','PRAIA DO CANTO','PRAIA DO SUÁ','ROMÃO','SANTA HELENA',
                       'SANTA LÚCIA','SANTA LUÍZA', 'ILHAS OCEÂNICAS DE TRINDADE']
}

# funções auxiliares
def regiao_saude(bairro):
    for regiao, bairros in regioes_saude.items():
        if bairro in bairros:
            return regiao

def get_renda(bairro):
    renda = []
    for idx, row in dados_renda_per_capta.iterrows():
        if bairro in row['NOME_UDH']:
            renda.append(row['RDPC'])
    if not renda:
        return 0
    return sum(renda)/len(renda)

In [ ]:
# remover linhas duplicadas
df.drop_duplicates(subset='AppointmentID', keep='first')

# alterar coluna "No-show" para algo mais intuitivo
df['No-show'].replace({'Yes': 0, 'No': 1}, inplace=True)
df.rename(columns={'No-show': 'Compareceu'}, inplace=True)

# corrigir gêneros
df['Gender'].replace({'Female':'F','Male':'M'}, inplace=True)

# corrigir bairros
df['Neighbourhood'].replace({'RESIST√äNCIA':'RESISTÊNCIA','REP√öBLICA':'REPÚBLICA','S√ÉO PEDRO':'SÃO PEDRO',
                             'S√ÉO CRIST√ìV√ÉO':'SÃO CRISTÓVÃO','MARU√çPE':'MARUÍPE','GRANDE VIT√ìRIA':'GRANDE VITÓRIA',
                             'S√ÉO BENEDITO':'SÃO BENEDITO','SANTO ANDR√â':'SANTO ANDRÉ','ANT√îNIO HON√ìRIO':'ANTÔNIO HONÓRIO',
                             'M√ÅRIO CYPRESTE':'MÁRIO CYPRESTE','SANTO ANT√îNIO':'SANTO ANTÔNIO','PRAIA DO SU√Å':'PRAIA DO SUÁ',
                             'ITARAR√â':'ITARARÉ','INHANGUET√Å':'INHANGUETÁ','UNIVERSIT√ÅRIO':'UNIVERSITÁRIO',
                             'S√ÉO JOS√â':'SÃO JOSÉ','REDEN√á√ÉO':'REDENÇÃO','CARATO√çRA':'CARATOÍRA',
                             'ILHA DO PR√çNCIPE': 'ILHA DO PRÍNCIPE','ROM√ÉO': 'ROMÃO', 'FORTE S√ÉO JO√ÉO': 'FORTE SÃO JOÃO',
                             'JOANA D¬¥ARC': 'JOANA D\'ARC','CONSOLA√á√ÉO': 'CONSOLAÇÃO', 'SANTA CEC√çLIA': 'SANTA CECÍLIA',
                             'SANTA L√öCIA': 'SANTA LÚCIA','SANTA LU√çZA': 'SANTA LUÍZA', 'SEGURAN√áA DO LAR': 'SEGURANÇA DO LAR',
                             'Tabuazerio': 'TABUAZEIRO','ILHAS OCE√ÇNICAS DE TRINDADE': 'ILHAS OCEÂNICAS DE TRINDADE',
                             'ENSEADA DO SU√Å': 'ENSEADA DO SUÁ', 'DE LOURDE': 'DE LOURDES'}, inplace=True)
df['Neighbourhood'] = df.Neighbourhood.str.upper()

# adicionar macro-regiões de saúde
df['Regiao_Saude'] = df['Neighbourhood'].progress_apply(lambda x: regiao_saude(x))

# adicionar tempo entre 'dia que marcou a consulta' e 'dia da consulta'
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay']).dt.date
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay']).dt.date
df['time_delta'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

# adicionar renda média por bairro
df['Renda'] = df['Neighbourhood'].progress_apply(lambda x: get_renda(x))

# Percentual de comparecimento 

In [ ]:
foram_percent = df['Compareceu'].mean()
faltaram_percent = (100 - foram_percent)  

print(f'A assiduidade nas consultas foi de {foram_percent:.2f}%. Ausência de {faltaram_percent:.2f}%. ')

### Percentual de comparecimento após o dia 0

In [ ]:
# criando dataframe com os dias maiores que zero
df_sem_dia0 = df[df['time_delta'] > 0] 
presenca_sem_dia0 = len(df_sem_dia0[df_sem_dia0['Compareceu']==1])
# Verificando quantas consultas foram marcadas para o mesmo dia e criando um dataframe com essas consultas
consultas_mais_1dia=len(df_sem_dia0)
total_consultas=len(df['time_delta'])

print(f'Total de consultas marcadas com um ou mais de um dia de intervalo: {consultas_mais_1dia}.')
print(f'Total de consultas: {total_consultas}.')
print('\nRelação das pessoas que compareceram, com exceção do dia 0, à consulta (1 - Sim, 0 - Não):')
print(df_sem_dia0['Compareceu'].value_counts())
print(f'\nFormato do novo DataFrame: Linhas: {df_sem_dia0.shape[0]}, Colunas: {df_sem_dia0.shape[1]}')
print(f'Percentual de consultas marcadas para um intervalo de tempo maior que um dia: {consultas_mais_1dia/total_consultas*100:.2f}%.')
print(f'\n{presenca_sem_dia0} presenças')
print(f'Percentual de presença nessas consultas: {presenca_sem_dia0/len(df_sem_dia0)*100:.2f}%.')


### percentual de ausencia pelo sexo biológico, coluna 'Gender'


In [ ]:
df.groupby('Gender')['Compareceu'].mean()

### percentual de ausencia entre os benificiários do bolsa família


In [ ]:
# percentual de ausencia entre os benificiários do bolsa família
df.groupby('Scholarship')['Compareceu'].mean()

### percentual de ausência entre o grupo PCD


In [ ]:
df['Handcap'].replace(to_replace = (2 ,3 ,4), value = 1 , inplace = True)
#df['Handcap'].value_counts()

df.groupby('Handcap')['Compareceu'].mean() # bronze

### percentual de ausência entre os pacientes com problemas com álcool


In [ ]:
# percentual de ausência entre os alcoólótras
df.groupby('Alcoholism')['Compareceu'].mean()

### percentual de ausência entre os diabéticos


In [ ]:
# percentual de ausência entre os diabéticos
df.groupby('Diabetes')['Compareceu'].mean() # medalha de prata em assiduidade

### percentual de ausência entre os hipertensos


In [ ]:
# percentual de ausência entre os hipertensos
df.groupby('Hipertension')['Compareceu'].mean() # campeões em assiduidade

### Percentual de ausência entre os pacientes que receberam mensagem de lembrete pelo SMS


In [ ]:
# percentual de ausência entre os pacientes que receberam mensagem de lembrete pelo SMS
df.groupby('SMS_received')['Compareceu'].mean()

# percebe-se que quem recebe sms tende a faltar mais do que o grupo q não recebeu

## Faixa etária


In [ ]:
#criar uma coluna de faixa etária
df['Faixa etaria']=pd.cut(df['Age'], bins=[-1,12,18,30,60,125], labels=['Crianças','Adolescentes','Adultons Jovens','Adultos','Idosos'])


In [ ]:
# verifica criação de nova coluna faixa etária
df.head(10)

 ### Qual a faixa etária mais assídua e a mais ausente? 

In [ ]:
# agrupa em ordem decrescente a assiduidade por faixa etária
df.groupby('Faixa etaria')['Compareceu'].mean().sort_values(ascending=False)

### qual faixa etária é mais acometida com problemas com álcool? 


In [ ]:
df.groupby('Faixa etaria')['Alcoholism'].mean().sort_values(ascending=False)

### qual faixa etária é mais acometida pelo diabetes? 


In [ ]:
df.groupby('Faixa etaria')['Diabetes'].mean().sort_values(ascending=False)

### qual faixa etária é mais acometida pelo hipertensão? 

In [ ]:
df.groupby('Faixa etaria')['Hipertension'].mean().sort_values(ascending=False)

# Bairros: 

### Qual(ais) bairros mais assíduos ? 

In [ ]:
df.groupby('Neighbourhood')['Compareceu'].mean().sort_values(ascending=False)
# Os últimos são os bairros de pacientes com maior ausência em consultas

###  Qual(is) bairro(s) detém maior número de beneficiários do programa Bolsa família?


In [ ]:
df.groupby('Neighbourhood')['Scholarship'].mean().sort_values(ascending=False)


### Qual(is) bairro(s) detém maior número de pacientes com problemas com álcool? 

In [ ]:
df.groupby('Neighbourhood')['Alcoholism'].mean().sort_values(ascending=False)


### Qual(is) bairro(s) detém maior número de hipertensos? 

In [ ]:
df.groupby('Neighbourhood')['Hipertension'].mean().sort_values(ascending=False)


### Qual(is) bairro(s) detém maior número de diabéticos? 

In [ ]:
df.groupby('Neighbourhood')['Diabetes'].mean().sort_values(ascending=False)


## Em comparação ao grupo não-PCD, o grupo com deficiência compareceu mais nas consultas.

In [ ]:
df.groupby(['Handcap'])['Compareceu'].mean() 

### Quantas consultas foram marcadas para cada especialidade?

In [ ]:

df_pcd = df.loc[df['Handcap']==1]


hipertensos_pcd = df_pcd.loc[df_pcd['Hipertension']==1]
alcolismo_pcd = df_pcd.loc[df_pcd['Alcoholism']==1]
diabetes_pcd = df_pcd.loc[df_pcd['Diabetes']==1]

print(f'Foram {len(alcolismo_pcd)} consultas marcadas para Alcoolismo no grupo PCD')
print(f'Foram {len(diabetes_pcd)} consultas marcadas para Diabetes no grupo PCD')
print(f'Foram {len(hipertensos_pcd)} consultas marcadas para Hipertensão no grupo PCD')


### Qual gênero compareceu mais nas consultas marcadas no grupo PCD? Genero Masculino


In [ ]:
df_pcd.groupby('Gender')['Compareceu'].mean()

### Qual a faixa etária que mais compareceu nas consultas marcadas?

In [ ]:
df_pcd['Faixa etaria'] = pd.cut(df_pcd['Age'],bins=[-1,12,18,30,60,125],labels=['Crianças','Adolescentes','Jovens Adultos','Adultos','Idosos'])
df_pcd.groupby(['Faixa etaria'])['Compareceu'].mean().sort_values(ascending=False)

## No geral, incluindo TODAS as consultas, até as imediatas, receber o sms não impactou o grupo PCD, porém se isolarmos as consultas imediatas vemos que sim teve uma pequena diferença, e vamos ver isso mais a frente.

In [ ]:
df_pcd.groupby(['SMS_received'])['Compareceu'].mean()

## Alocando as respostas de comparecimentos em grupos de:
### Recebeu o sms - 1. Não recebeu o sms - 0.<br>

## E separando o grupo por consultas realizadas a partir de 3 dias de marcação temos os seguintes resultados:

### No grupo de consultas marcadas para "Diabetes" houve um aumento de 7% no comparecimento de quem recebeu o lembrete por sms.

In [ ]:

sms_diabetes_pcd = diabetes_pcd.loc[(diabetes_pcd['time_delta']>2)]
sms_diabetes_pcd.groupby(['SMS_received'])['Compareceu'].mean()

### No grupo de consultas marcadas para "Hipertensão" houve um aumento de 8% no comparecimento de quem recebeu o lembrete por sms. 

In [ ]:

sms_hipertensao = hipertensos_pcd.loc[(hipertensos_pcd['time_delta']>2)]
sms_hipertensao.groupby(['SMS_received'])['Compareceu'].mean()

### No grupo de consultas marcadas para "Alcoolismo" receber SMS não fez diferença.


In [ ]:
sms_alcolismo_pcd = alcolismo_pcd.loc[(alcolismo_pcd['time_delta']>2)]
sms_alcolismo_pcd.groupby(['SMS_received'])['Compareceu'].mean()

### Será que com o tempo mais longo de espera de consulta há mais ausência de deficientes? 
### O nível de comparecimento é alto nos primeiros dias, mas quanto mais longe da data a consulta é marcada, as taxas de comparecimento diminuem.

In [ ]:
df_pcd.pivot_table(index='time_delta', aggfunc='count', values='Compareceu').plot(kind='bar', xticks= (np.arange(0,61,30)),figsize=(15,5), title = 'Comparecimento por tempo de espera pela consulta', xlabel = 'Dias depois da marcação')

#### Solução: Quanto mais cedo as consultas forem marcadas, e tendo sido reforçados os lembretes por sms, a taxa de AUSÊNCIA do grupo PCD diminuirá.

# Mateus 

In [ ]:
df_3plus = df[df['time_delta'] > 3]
df_3plus.head()

In [ ]:
df_3plus['Neighbourhood'].value_counts().plot(kind='bar', figsize=(20, 6))
plt.show()

In [ ]:
df_3plus['Compareceu'].value_counts()


In [ ]:
df_3plus['Compareceu'].value_counts().plot(kind='bar')
plt.show()


In [ ]:
df_3plus['Age'].hist(bins=50)
plt.show()


In [ ]:
df_3plus['Gender'].value_counts().plot(kind='pie', autopct='%1.1f%%')
plt.show()


In [ ]:
df_3plus.groupby('SMS_received')['Compareceu'].count().reset_index()

In [ ]:
df_3plus.groupby(['SMS_received','Compareceu'])['PatientId'].count().reset_index()

In [ ]:
df_3plus.groupby('AppointmentDayOfWeek')['Compareceu'].mean().sort_values(ascending=False)

In [ ]:
df_3plus.groupby('ScheduledDayOfWeek')['Compareceu'].mean().sort_values(ascending=False)

In [ ]:
df_3plus['ScheduledDayOfWeek'].value_counts().plot(kind='bar')

In [ ]:
df_3plus['AppointmentDayOfWeek'].value_counts().plot(kind='bar')


In [ ]:
sms_received_by_gender = df_3plus.groupby(['Gender', 'SMS_received'])['Compareceu'].mean().reset_index()

In [ ]:
sms_received_by_gender

In [ ]:
# Agrupar por faixa etária e comparecimento
age_by_show = df_3plus.groupby(['Faixa etaria', 'Compareceu'])['PatientId'].count().unstack()

# Criar subplot para cada faixa etária
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15,10))

# Título principal
fig.suptitle('Porcentagem de pacientes por faixa etária e comparecimento', fontsize=20)

# Loop pelas faixas etárias
for i, (index, data) in enumerate(age_by_show.iterrows()):
    # Calcular porcentagem de comparecimento
    total = data.sum()
    compareceu = data[1]
    porcentagem = compareceu / total * 100
    
    # Definir posição do subplot
    row = i // 3
    col = i % 3
    ax = axes[row, col]
    
    # Plotar gráfico de pizza
    data.plot(kind='pie', 
              ax=ax, 
              autopct='%1.1f%%', 
              labels=['Não Compareceu', 'Compareceu'],
              title=f'{index} ({porcentagem:.1f}% compareceu)')
    
# Ajustar espaçamento entre subplots
plt.subplots_adjust(hspace=0.5, wspace=0.5)

# Exibir gráfico
plt.show()


<h1>Resultados do envio de SMS</h1>
<h3>Introdução</h3>
O envio de SMS para lembrar os pacientes de consultas médicas pode ser uma forma eficaz de melhorar a adesão ao tratamento e, consequentemente, a saúde do paciente. Nesta apresentação, iremos analisar os resultados da nossa campanha de envio de SMS para pacientes em Vitória-ES que têm consultas agendadas.

<h3>Dados</h3>
Os dados utilizados neste projeto foram obtidos a partir da base de dados 'Brazil's Medical Appointment - Show/No Show' disponível em https://www.kaggle.com/datasets/sarvaninandipati/brazils-medical-appointment-show-or-no-show. Nele existe a coluna de SMS e podemos relacionar com a coluna de comparecimento para analisar a eficácia dos sms.

<h3>Processos</h3>
Inicialmente, os dados mostravam que o SMS diminuia a taxa de comparecimento. Notamos que as consultas marcadas e realizadas no mesmo dia puxavam a taxa de comparecimento muito pra cima(aproximadamente 95% de comparecimento), e que as pessoas só começavam a receber SMS a partir de 3 dias de diferença da data da marcação para a realização da consulta. Então decidimos isolar esse caso pra fazer o estudo somente em pacientes que tenham um tempo de marcação e realização de consulta maior que 3 dias.


In [ ]:
df_3plus = df[df['time_delta'] > 3]
df_3plus.head()





<h3>Resultados</h3>
Os resultados mostram que a taxa de comparecimento dos pacientes que receberam o SMS foi 72,26%, enquanto a taxa de comparecimento dos pacientes que não receberam o SMS foi 66,65%.


In [ ]:
df_3plus.groupby('SMS_received')['Compareceu'].mean().reset_index()


Além disso, pudemos observar uma correlação entre o envio de SMS e a taxa de comparecimento dos pacientes. [Gráfico de barras ou outro tipo de gráfico que ilustre a correlação].


In [ ]:
import matplotlib.pyplot as plt

sms_comparecimento = df_3plus.groupby('SMS_received')['Compareceu'].mean().reset_index()

plt.bar(sms_comparecimento['SMS_received'], sms_comparecimento['Compareceu'])
plt.xlabel('Recebimento de SMS')
plt.ylabel('Taxa de comparecimento')
plt.xticks([0, 1], ['Não', 'Sim'])
plt.title('Relação entre recebimento de SMS e comparecimento')
plt.show()


A taxa de comparecimento aumenta em todas as faixas etárias, porém precisamos achar algo pra aliar ao sms e aumentar o comprometimento dos adolescentes e adultos jovens. E a taxa para crianças é a mais beneficiada pelo uso do SMS. Sendo assim, é necessário usar a estratégia para princilpamente Crianças, Adolescentes e Adultos jovens.

In [ ]:
sms_by_age_show = df_3plus.groupby(['Faixa etaria', 'SMS_received'])['Compareceu'].mean().reset_index()
sms_by_age_show

A taxa de comparecimento dos homens é mais afetada que a das mulheres com o uso do SMS, a porcentagem de homens que comparecem com o recebimento do SMS fica levemente maior que a das mulheres, note que os homens tem uma taxa de comparecimento menor no geral.

In [ ]:
sms_received_by_gender = df_3plus.groupby(['Gender', 'SMS_received'])['Compareceu'].mean().reset_index()
sms_received_by_gender

In [ ]:
sms_received_by_gender = df_3plus.groupby(['Gender'])['Compareceu'].mean().reset_index()
sms_received_by_gender




É importante ressaltar que, embora esses cálculos possam nos dar uma ideia geral da eficácia do SMS, há muitos outros fatores que podem influenciar a decisão de um paciente de comparecer a uma consulta, como a gravidade da condição médica, o acesso ao transporte, o nível socioeconômico, entre outros. Portanto, esses cálculos devem ser interpretados com cautela e considerando o contexto.

<h3>Conclusão</h3>
Com base nos resultados da nossa análise, podemos concluir que o envio de SMS é uma estratégia eficaz para melhorar a adesão dos pacientes a consultas médicas, sendo ainda mais recomendada para homens, adultos jovens, adolescentes e pais de crianças. Recomendamos que esta estratégia seja adotada por outras instituições médicas em todo o país para melhorar a saúde da população e reduzir os custos com tratamentos médicos desnecessários.

# Vinícius

# Avaliação da presença nos bairros

Primeiro, removemos as consultas que foram realizadas no mesmo dia, pois queremos ver a presença em consultas futuras.

In [ ]:
not_emergency = df.query('time_delta > 0')

Após isso, separamos as consultas por bairros, e verificamos a média de comparecimento por bairro.

In [ ]:
not_emergency.groupby(['Neighbourhood', 'Renda'])['Compareceu'].mean().sort_values(ascending=True)*100

É possível ver uma tendência de que bairros mais pobres tem uma taxa de ausência maior nas consultas.

Podemos colocar essa representação visualmente em gráficos:

In [ ]:
b_r_p = not_emergency.groupby(['Neighbourhood', 'Renda'])['Compareceu'].mean().sort_values(ascending=True)*100
x = [b[0] for b in b_r_p.index]
y = [b[1] for b in b_r_p.index]
plt.figure(figsize=(20,2))
plt.subplot(151)
plt.plot(x, b_r_p.values)
plt.subplot(152)
plt.plot(x, y, )
plt.show()

Fazendo a separação de acordo com as Regiões de Saúde, temos:

In [ ]:
not_emergency.groupby('Regiao_Saude')['Renda'].mean().sort_values(ascending=True)

# Aline

# Data de agendamento:
O objetivo é avaliar a relação entra a data de agendamento e a assiduidade dos pacientes, visando criar um indicador para estabelecimento de um perfil propenso a não comparecer as consultas

In [ ]:
df_aux = pd.read_excel('appointment_prediction_data.xlsx')
df_aux['No-show'].replace({'Yes': 0, 'No': 1}, inplace=True)

df_aux.rename(columns={'No-show': 'Compareceu'}, inplace=True)

In [ ]:
df_aux

In [ ]:
df_aux['ScheduledDay'] = pd.to_datetime(df_aux['ScheduledDay'])
df_aux['ScheduledDay_aux'] = pd.to_datetime(df_aux['ScheduledDay']).dt.date
df_aux['AppointmentDay_aux'] = pd.to_datetime(df_aux['AppointmentDay']).dt.date
df_aux['time_delta'] = (df_aux['AppointmentDay_aux'] - df_aux['ScheduledDay_aux']).dt.days
df_aux

In [ ]:
#Extração da hora em que foi agendada a consulta
df_aux['ScheduledHour'] = df_aux.ScheduledDay.dt.strftime("%H")
df_aux

Com base no gráfico abaixo é possível notar que pessoas que agendaram suas consultas no perído da manhã foram as mais assíduas

In [ ]:
#Verificação da assiduidade por hora
df_aux.groupby('ScheduledHour')['Compareceu'].mean().sort_values(ascending=False).plot()

Com base no gráfico abaixo é possível notar que pessoas que agendaram suas consultas no sábado foram as mais assíduas

In [ ]:
#Extração do dia da semana que foram realizadas os agendamentos
df_aux['ScheduledWeekDay'] = df_aux.ScheduledDay.dt.day_name()
df_aux.groupby('ScheduledWeekDay')['Compareceu'].mean().sort_values(ascending=False).plot()

Afim de validar se o comportamento se repete desocnsiderando possíveis casos de emergências (consultas agendadas e realizadas no mesmo dia), refizemos a análise eliminando esses casos e podemos notar que o mesmo se repete

In [ ]:
df_filtrado = df_aux[df_aux.time_delta > 0].copy()

In [ ]:
#Extração da hora em que foi agendada a consulta
df_filtrado['ScheduledHour'] = df_filtrado.ScheduledDay.dt.strftime("%H")
df_filtrado.groupby('ScheduledHour')['Compareceu'].mean().sort_values(ascending=False).plot()

In [ ]:
df_filtrado['ScheduledWeekDay'] = df_filtrado.ScheduledDay.dt.day_name()
df_filtrado.groupby('ScheduledWeekDay')['Compareceu'].mean().sort_values(ascending=False).plot()

# Felipe

# Análise do tempo entre a marcação da consulta e a consulta em si

- Felipe Cunha

### Hipóteses e pontos a serem investigados: 
- Uma parte das consultas foi marcada para o mesmo dia e a taxa de comparecimento deve ser mais alta;
- Quanto maior a diferença de tempo entre a marcação da consulta e a consulta, maior a taxa de ausentes;
- Quanto maior a diferença de tempo, mais as pessoas recebem SMS para lembrar da consulta;
    - E nesse caso, o recebimento de SMS faz com que aumente a taxa de presença?
- Verificar também as outras variáveis em relação a faixas de intervalo de tempo

### Análise das consultas marcadas para o mesmo dia

In [ ]:
# Verificando quantas consultas foram marcadas para o mesmo dia e criando um dataframe com essas consultas
consultas_mesmo_dia = df['time_delta'].value_counts()[0]
total_consultas = len(df['time_delta'])

df_dia0 = df[df['time_delta'] == 0]
df_dia0['Compareceu'].value_counts()

print(f'Total de consultas marcadas para o mesmo dia: {consultas_mesmo_dia}.')
print(f'Total de consultas: {total_consultas}.')
print('\nRelação das pessoas que compareceram à consulta (1 - Sim, 0 - Não):')
print(df_dia0['Compareceu'].value_counts())
print(f'\nFormato do novo DataFrame: Linhas: {df_dia0.shape[0]}, Colunas: {df_dia0.shape[1]}')

In [ ]:
presenca_dia0 = df_dia0['Compareceu'].value_counts()[1] # Relação de pessoas que foram a consulta
ausencia_dia0 = df_dia0['Compareceu'].value_counts()[0] # Relação de pessoas que não foram a consulta

print(f'Percentual de consultas marcadas para o mesmo dia: {consultas_mesmo_dia/total_consultas*100:.2f}%.')
print(f'Percentual de presença nessas consultas: {presenca_dia0/len(df_dia0)*100:.2f}%.')

In [ ]:
df_dia0.groupby('Compareceu').mean() # Panorama geral dos dados para as consultas marcadas no mesmo dia

##### Conclusões
- Concluí-se que nesse tipo de consulta a taxa de comparecimento é bem alta (> 95%);
- A média de idade para quem compareceu é de 35 anos, um pouco maior que a das pessoas que não compareceram (28 anos);
- Fatores como receber ou não o bolsa família, doenças pré-existentes não parecem influenciar na presença ou ausência nas consultas;

### Análise do restante dos dados, consultas com 1 ou mais dias de intervalo entre a marcação e a consulta

In [ ]:
df_sem_dia0 = df[df['time_delta'] > 0] # criando o dataframe
df_sem_dia0.head()

In [ ]:
# Verificando quantas consultas foram marcadas com mais de um dia de intervalo e criando um dataframe com essas consultas
consultas_mais_1dia = len(df_sem_dia0)

print(f'Total de consultas marcadas com um ou mais de um dia de intervalo: {consultas_mais_1dia}.')
print(f'Total de consultas: {total_consultas}.')
print('\nRelação das pessoas que compareceram à consulta (1 - Sim, 0 - Não):')
print(df_sem_dia0['Compareceu'].value_counts())
print(f'\nFormato do novo DataFrame: Linhas: {df_sem_dia0.shape[0]}, Colunas: {df_sem_dia0.shape[1]}')

In [ ]:
presenca_sem_dia0 = df_sem_dia0['Compareceu'].value_counts()[1] # Relação de pessoas que foram a consulta

print(f'Percentual de consultas marcadas para um intervalo de tempo maior que um dia: {consultas_mais_1dia/total_consultas*100:.2f}%.')
print(f'Percentual de presença nessas consultas: {presenca_sem_dia0/len(df_sem_dia0)*100:.2f}%.')

##### Percebe-se uma queda em mais de 20% para consultas agendadas para dias posteriores. 
---
##### Em seguida foi verificado o comportamento da variável relativa ao intervalo de tempo

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(df_sem_dia0['time_delta'], bins=40) # histograma para ver a distribuição dos dados
plt.xlabel('Intervalo de Tempo entre a marcação e a consulta')
plt.ylabel('Quantidade de Consultas')
plt.title('Distribuição das consultas em diferentes intervalos de tempo.')
plt.show()

- A partir do histograma, percebeu-se que a maioria das consultas estava concentrada em um intervalo de tempo de até 40 dias. 
- Em seguida foi verificado se haviam outliers para essa série de dados

In [ ]:
plt.boxplot(df_sem_dia0['time_delta']) # boxplot para verificar a presença de outliers
plt.xlabel('Intervalo de Tempo')
plt.ylabel('Intervalo de Tempo em dias')
plt.show()

In [ ]:
q_high = df_sem_dia0['time_delta'].quantile(0.95) # Cálculo do limite máximo para retirada dos outliers em 95% de confiança
q_high

- Optou-se por retirar as consultas marcadas no mesmo dia e as com mais de 48 dias de intervalo para se fazer um estudo

In [ ]:
df_filtrado = df_sem_dia0[df_sem_dia0['time_delta'] < q_high] # criação do novo dataframe filtrado, sem outliers
df_filtrado.head() 

In [ ]:
print(f'Formato do DF com outliers: Linhas: {df_sem_dia0.shape[0]}, Colunas: {df_sem_dia0.shape[1]}')
print(f'Formato do DF sem outliers: Linhas: {df_filtrado.shape[0]}, Colunas: {df_filtrado.shape[1]}')
print(f'Foram retiradas {df_sem_dia0.shape[0] - df_filtrado.shape[0]} consultas.')

In [ ]:
# cálculo do total de pessoas que compareceram e das que não compareceram 
compareceu = df_filtrado[df_filtrado['Compareceu'] == 1]['time_delta']
n_compareceu = df_filtrado[df_filtrado['Compareceu'] == 0]['time_delta']

plt.figure(figsize=(10,5))
plt.hist(compareceu, bins=20, label='Compareceu')
plt.hist(n_compareceu, bins=20, label='Não compareceu')
plt.xlabel('Intervalo de Tempo entre a marcação e a consulta')
plt.ylabel('Quantidade de Consultas')
plt.title('Gráfico relacionando a presença e ausência dos pacientes em diferentes intervalos de tempo.')
plt.xticks(np.arange(0, 50, 7))
plt.legend()
plt.show()

- Percebe-se que a partir de um intervalo de tempo ~14 dias, ocorre uma maior taxa de ausentes às consultas. 

In [ ]:
df_filtrado.groupby('Compareceu').mean() # consultas com 1 ou mais dias de intervalo de tempo

In [ ]:
df_mais14 = df_filtrado[df_filtrado['time_delta'] >= 14] # consultas só com mais de 14 dias
df_mais14.groupby('Compareceu').mean()

- Nesses conjuntos analisados, percebe-se que a média de idades dos faltantes é menor do que a de pessoas que compareceram às consultas (em torno de 10% menores).
---
- Por fim, analisou-se apenas os "outliers", ou seja, consultas com mais de 48 dias entre a marcação e a consulta em si

In [ ]:
df_filtrado_2 = df_sem_dia0[df_sem_dia0['time_delta'] >= q_high] # criação do novo dataframe filtrado só dos outliers
df_filtrado_2.head() 

In [ ]:
print(f'Formato do DF com outliers: Linhas: {df_filtrado_2.shape[0]}, Colunas: {df_filtrado_2.shape[1]}')

In [ ]:
df_filtrado_2.groupby('Compareceu').mean() # consultas com 1 ou mais dias de intervalo de tempo

In [ ]:
f2_comparecimento = len(df_filtrado_2[df_filtrado_2['Compareceu'] == 1])

print(f'Percebe-se uma taxa de comparecimento nessas consultas de: {(f2_comparecimento/len(df_filtrado_2)*100):.2f}%.')

- Por fim, concluí-se que: 
    - As pessoas vão mais às consultas marcadas para o mesmo dia (~95%);
    - Essa taxa possui uma **tendência de queda** com o __aumento do intervalo de tempo__ entre a marcação e a consulta;
    - Para consultas com mais de **14 dias**, há uma **contribuição maior** do recebimento de SMS como alerta;
    - Em geral, a média de idade das pessoas que comparecem às consultas é maior;
    - Há uma tendência de usuários do bolsa família faltarem mais às consultas; 
    - Fatores como deficiência não parecem influenciar no comparecimento ou não às consultas.